In [296]:
import urllib.request
from bs4 import BeautifulSoup, Comment
import string
import pandas as pd

In [228]:
baseURL = "https://www.basketball-reference.com"
url="https://www.basketball-reference.com/players/"

In [184]:
alphabet=string.ascii_lowercase
playerURL=[]
playerID=[]
playerName=[]
yearStart=[]
yearEnd=[]
position=[]
height=[]
weight=[]
birthday=[]
college=[]

In [185]:
for i in range(0,26):
    playerURL.append(url+alphabet[i])
del playerURL[23]

In [186]:
for i in range(0,25):
    x = urllib.request.urlopen(playerURL[i]).read()
    soup = BeautifulSoup(x, "lxml")
    playerTag=soup.findAll(attrs={"data-stat" : "player"})
    YearMinTag=soup.findAll(attrs={"data-stat" : "year_min"})
    YearMaxTag=soup.findAll(attrs={"data-stat" : "year_max"})
    positionTag=soup.findAll(attrs={"data-stat" : "pos"})
    heightTag=soup.findAll(attrs={"data-stat" : "height"})
    weightTag=soup.findAll(attrs={"data-stat" : "weight"})
    bdayTag=soup.findAll(attrs={"data-stat" : "birth_date"})
    collegeTag=soup.findAll(attrs={"data-stat" : "college_name"})

    for i in range(1, len(playerTag)):
        playerID.append(playerTag[i].attrs["data-append-csv"])
        playerName.append(playerTag[i].a.contents[0])
        yearStart.append(YearMinTag[i].contents[0])
        yearEnd.append(YearMaxTag[i].contents[0])
        if(not positionTag[i].contents):
            position.append("")
        else:
            position.append(positionTag[i].contents[0])
        if(not heightTag[i].contents):
            height.append("")
        else:
            height.append(heightTag[i].contents[0])
        if(not weightTag[i].contents):
            weight.append("")
        else:
            weight.append(weightTag[i].contents[0])
        if(not bdayTag[i].contents):
            birthday.append("")
        else:
            birthday.append(bdayTag[i].a.contents[0])
        if(not collegeTag[i].contents):
            college.append("")
        else:
            college.append(collegeTag[i].a.contents[0])


In [187]:
labels = ['Player_ID', 'Name', 'YrStart', 'YrEnd', 'Pos', 'Height', 'Weight', 'Birth_date', 'College']
player_df = pd.DataFrame({"Player_ID": playerID,
                         "Name": playerName,
                         "YrStart": yearStart,
                         "YrEnd": yearEnd,
                         "Pos": position,
                         "Height": height,
                         "Weight": weight,
                         "Birth_date" : birthday,
                         "College" : college})
player_df=player_df[labels]

#fill NAs?

In [189]:
player_df.to_csv("NBA_Players.csv")

In [192]:
#Box Score Scrape
box_score_URL = "https://www.basketball-reference.com/boxscores/"

In [257]:
def gameBoxScoreLinks(a, b, c):
    #arguments represent date (a->month (MM), b-> day (DD), c-> year(YYYY))
    #if month or day is less than 10, write 5 as "5" not "05"
    #returns links to box score game links
    urlBox=[]
    dateURL = "?month=" + str(a) + "&day=" + str(b) +"&year=" + str(c)
    gameURL = box_score_URL + dateURL
    x = urllib.request.urlopen(gameURL).read()
    gameSoup = BeautifulSoup(x, "lxml")
    gameTag = gameSoup.findAll("td", {"class" : "right gamelink"})
    
    for i in range(0,len(gameTag)):
        urlBox.append(baseURL + gameTag[i].a["href"])
        
    return urlBox

In [258]:
urlBox=gameBoxScoreLinks(2,7,2009)

In [454]:
def gameBoxScore(urlBox):
    #takes a list of urls of box scores
    team1=[]
    team2=[]
    date=[]
    playerID=[]
    minutes=[]
    fg=[]
    fga=[]
    fgpct=[]
    fg3 = []
    fg3a=[]
    fg3pct=[]
    ft=[]
    fta=[]
    ftpct=[]
    orb=[]
    drb=[]
    trb=[]
    ast=[]
    stl=[]
    blk=[]
    tov=[]
    pf=[]
    pts=[]
    plus_minus=[]
    ts_pct=[]
    efg_pct=[]
    off_rtg=[]
    def_rtg=[]
    finalMinutes=[]
    for i in range(0, len(urlBox)):
        a = urllib.request.urlopen(urlBox[i]).read()
        boxSoup = BeautifulSoup(a, "lxml")
        headerTag = boxSoup.findAll("th", {"class": "left"})
        for i in range(0,len(headerTag)):
            if(headerTag[i].has_attr("data-append-csv")):
                if(headerTag[i]["data-append-csv"] not in playerID):
                    playerID.append(headerTag[i]["data-append-csv"])
                    date.append(boxSoup.title.contents[0].split(", ")[1] + boxSoup.title.contents[0].split(",")[2].split(" |")[0])
                    team1.append(boxSoup.title.contents[0].split(" at ")[0])
                    team2.append(boxSoup.title.contents[0].split(" at ")[1].split(" ")[0] + " " + boxSoup.title.contents[0].split(" at ")[1].split(" ")[1])
        minuteTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "mp"})
        for i in range(0,len(minuteTag)):
            if(minuteTag[i].parent.parent.name !="tfoot"):
                if(minuteTag[i].contents[0] not in minutes):
                    if(not minuteTag[i].contents):
                        minutes.append("")
                    else:
                        minutes.append(minuteTag[i].contents[0])  
        for i in range(0, len(minutes)):
            finalMinutes.append(minutes[i])
            
        fgTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "fg"})
        for i in range(0,len(fgTag)):
            if(fgTag[i].parent.parent.name != "tfoot"):
                if(not fgTag[i].contents):
                    fg.append("")
                else:
                    fg.append(fgTag[i].contents[0]) 
        fgaTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "fga"})
        for i in range(0,len(fgaTag)):
            if(fgaTag[i].parent.parent.name!="tfoot"):
                if(fgaTag[i].contents not in fga):
                    if(not fgaTag[i].contents):
                        fga.append("")
                    else:
                        fga.append(fgTag[i].contents[0]) 
        fgpctTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "fg_pct"})
        for i in range(0,len(fgpctTag)):
            if(fgpctTag[i].parent.parent.name!="tfoot"):
                if(not fgpctTag[i].contents):
                    fgpct.append("")
                else:
                    fgpct.append(fgpctTag[i].contents[0])
        fg3Tag=boxSoup.findAll("td", {"class": "right ", "data-stat": "fg3"})
        for i in range(0,len(fg3Tag)):
            if(fg3Tag[i].parent.parent.name!="tfoot"):
                if(not fg3Tag[i].contents):
                    fg3.append("")
                else:
                    fg3.append(fg3Tag[i].contents[0])
                
        fg3aTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "fg3a"})
        for i in range(0,len(fg3aTag)):
            if(fg3aTag[i].parent.parent.name!="tfoot"):
                if(not fg3aTag[i].contents):
                    fg3a.append("")
                else:
                    fg3a.append(fg3aTag[i].contents[0])
        fg3pctTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "fg3_pct"})
        for i in range(0,len(fg3pctTag)):
            if(fg3pctTag[i].parent.parent.name!="tfoot"):
                if(not fg3pctTag[i].contents):
                    fg3pct.append("")
                else:
                    fg3pct.append(fg3pctTag[i].contents[0])
        ftTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "ft"})
        for i in range(0,len(ftTag)):
            if(ftTag[i].parent.parent.name!="tfoot"):
                if(not ftTag[i].contents):
                    ft.append("")
                else:
                    ft.append(ftTag[i].contents[0])  
        ftaTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "fta"})
        for i in range(0,len(ftaTag)):
            if(ftaTag[i].parent.parent.name!="tfoot"):
                if(not ftaTag[i].contents):
                    fta.append("")
                else:
                    fta.append(ftaTag[i].contents[0])
        ftpctTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "ft_pct"})
        for i in range(0,len(ftpctTag)):
            if(ftpctTag[i].parent.parent.name!="tfoot"):
                if(not ftpctTag[i].contents):
                    ftpct.append("")
                else:
                    ftpct.append(ftpctTag[i].contents[0])
        orbTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "orb"})
        for i in range(0,len(orbTag)):
            if(orbTag[i].parent.parent.name!="tfoot"):
                if(not orbTag[i].contents):
                    orb.append("")
                else:
                    orb.append(orbTag[i].contents[0])
        drbTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "drb"})
        for i in range(0,len(drbTag)):
            if(drbTag[i].parent.parent.name!="tfoot"):
                if(not drbTag[i].contents):
                    drb.append("")
                else:
                    drb.append(drbTag[i].contents[0])
        trbTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "trb"})
        for i in range(0,len(trbTag)):
            if(trbTag[i].parent.parent.name!="tfoot"):
                if(not trbTag[i].contents):
                    trb.append("")
                else:
                    trb.append(trbTag[i].contents[0])
        astTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "ast"})
        for i in range(0,len(astTag)):
            if(astTag[i].parent.parent.name!="tfoot"):
                if(not astTag[i].contents):
                    ast.append("")
                else:
                    ast.append(astTag[i].contents[0])
        stlTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "stl"})
        for i in range(0,len(stlTag)):
            if(stlTag[i].parent.parent.name!="tfoot"):
                if(not stlTag[i].contents):
                    stl.append("")
                else:
                    stl.append(stlTag[i].contents[0])    
        blkTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "blk"})
        for i in range(0,len(blkTag)):
            if(blkTag[i].parent.parent.name!="tfoot"):
                if(not blkTag[i].contents):
                    blk.append("")
                else:
                    blk.append(blkTag[i].contents[0])
        tovTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "tov"})
        for i in range(0,len(tovTag)):
            if(tovTag[i].parent.parent.name!="tfoot"):
                if(not tovTag[i].contents):
                    tov.append("")
                else:
                    tov.append(tovTag[i].contents[0])
        pfTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "pf"})
        for i in range(0,len(pfTag)):
            if(pfTag[i].parent.parent.name!="tfoot"):
                if(not pfTag[i].contents):
                    pf.append("")
                else:
                    pf.append(pfTag[i].contents[0])
        ptsTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "pts"})
        for i in range(0,len(ptsTag)):
            if(ptsTag[i].parent.parent.name!="tfoot"):
                if(not ptsTag[i].contents):
                    pts.append("")
                else:
                    pts.append(ptsTag[i].contents[0])    
        plus_minusTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "plus_minus"})
        for i in range(0,len(plus_minusTag)):
            if(plus_minusTag[i].parent.parent.name!="tfoot"):
                if(not plus_minusTag[i].contents):
                    plus_minus.append("")
                else:
                    plus_minus.append(plus_minusTag[i].contents[0])
        ts_pctTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "ts_pct"})
        for i in range(0,len(ts_pctTag)):
            if(ts_pctTag[i].parent.parent.name!="tfoot"):
                if(not ts_pctTag[i].contents):
                    ts_pct.append("")
                else:
                    ts_pct.append(ts_pctTag[i].contents[0])
        efg_pctTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "efg_pct"})
        for i in range(0,len(efg_pctTag)):
            if(efg_pctTag[i].parent.parent.name!="tfoot"):
                if(not efg_pctTag[i].contents):
                    efg_pct.append("")
                else:
                    efg_pct.append(efg_pctTag[i].contents[0])
        off_rtgTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "off_rtg"})
        for i in range(0,len(off_rtgTag)):
            if(off_rtgTag[i].parent.parent.name!="tfoot"):
                if(not off_rtgTag[i].contents):
                    off_rtg.append("")
                else:
                    off_rtg.append(off_rtgTag[i].contents[0])
        def_rtgTag=boxSoup.findAll("td", {"class": "right ", "data-stat": "def_rtg"})
        for i in range(0,len(def_rtgTag)):
            if(def_rtgTag[i].parent.parent.name!="tfoot"):
                if(not def_rtgTag[i].contents):
                    def_rtg.append("")
                else:
                    def_rtg.append(def_rtgTag[i].contents[0])
        minutes=[]
    labels=["Team1", "Team2", "Date", "PlayerID", "Minutes", "FG", "FGA", "FGPCT", "3PT", "3PTA", "3PTPCT", "FT", "FTA",
            "FTPCT", "ORB", "DRB", "TRB", "AST", "STL", "BLK", "TOV", "PF", "PTS", "+/-", "TSPCT", "EFGPCT", "OFFRTG", "DEFRTG"]
    
    boxScore_df=pd.DataFrame({"Team1": team1,
                              "Team2": team2,
                              "Date": date,
                              "PlayerID": playerID,
                              "Minutes": finalMinutes,
                              "FG": fg,
                              "FGA": fga,
                              "FGPCT":fgpct,
                              "3PT": fg3,
                              "3PTA": fg3a,
                              "3PTPCT": fg3pct,
                              "FT": ft,
                              "FTA": fta,
                              "FTPCT": ftpct,
                              "ORB": orb,
                              "DRB": drb,
                              "TRB": trb,
                              "AST": ast,
                              "STL": stl,
                              "BLK": blk,
                              "TOV": tov,
                              "PF": pf,
                              "PTS": pts,
                              "+/-": plus_minus,
                              "TSPCT":ts_pct,
                              "EFGPCT":efg_pct,
                              "OFFRTG": off_rtg,
                              "DEFRTG": def_rtg
        })
    boxScore_df = boxScore_df[labels]


    return boxScore_df

In [455]:
boxScore_df=gameBoxScore(urlBox)

In [456]:
boxScore_df.head()

,Team1,Team2,Date,PlayerID,Minutes,FG,FGA,FGPCT,3PT,3PTA,...,STL,BLK,TOV,PF,PTS,+/-,TSPCT,EFGPCT,OFFRTG,DEFRTG
0,Los Angeles Clippers,Atlanta Hawks,February 7 2009,randoza01,36:12,10,10,.556,3,3,...,1,0,0,2,25,+20,.662,.639,138,107
1,Los Angeles Clippers,Atlanta Hawks,February 7 2009,thornal01,35:46,13,13,.591,1,2,...,1,1,1,2,31,+21,.640,.614,131,105
2,Los Angeles Clippers,Atlanta Hawks,February 7 2009,gordoer01,32:55,5,5,.500,1,3,...,0,0,0,3,11,+5,.550,.550,118,112
3,Los Angeles Clippers,Atlanta Hawks,February 7 2009,cambyma01,30:29,6,6,.857,0,0,...,0,0,0,2,16,+13,.870,.857,188,110
4,Los Angeles Clippers,Atlanta Hawks,February 7 2009,davisba01,26:17,6,6,.400,1,5,...,2,0,1,3,13,+4,.409,.433,109,104


In [458]:
boxScore_df.to_csv("BoxScore02072009.csv")